# The Game 
## without interactions with the map or the google sheet

initial conditions with (until now) random numbers:

In [1]:
import numpy as np
import math

In [2]:
# create dummy matrix
n_plots = 4
rows = cols = 80

plot_length = int(rows/n_plots)
dummy_plot = np.ones(plot_length**2).reshape((plot_length, plot_length))
A1 = dummy_plot
A2 = dummy_plot * 2
B1 = dummy_plot * 3
B2 = dummy_plot * 4
large_matrix = np.block([[A1, A2], [B1, B2]])
large_dummy_matrix = np.ones_like(large_matrix)
n = plot_length
coef_matrix = np.array([[1, 2], [3, 4]])
result = np.multiply(large_dummy_matrix, np.kron(coef_matrix, np.ones((n,n))))
matrix_indizes = np.indices((n_plots, n_plots), dtype="uint8") + 1
row_indizes, column_indizes = matrix_indizes[0], matrix_indizes[1]
plot_definition_matrix = np.char.add(row_indizes.astype(np.str), column_indizes.astype(np.str)).astype(np.uint8)
dummy_playing_field_matrix = np.ones(shape=(rows, cols), dtype=np.uint8)
large_plot_definition_matrix = np.multiply(
    dummy_playing_field_matrix,
    np.kron(plot_definition_matrix, np.ones(shape=(plot_length, plot_length)))
)

In [3]:
# what I used from your matrix-dummies-notebook:
lulc_matrix = dummy_playing_field_matrix             # 80 x 80
cooperation_matrix = dummy_playing_field_matrix      # 80 x 80
plot_definition_matrix                               # 4 x 4
tourism_matrix = plot_definition_matrix              # 4 x 4
n_blocks = 4
n_pixel = 20

# other assumtions:
rounds = 10   # number of rounds
teams = 3    # in round 3 - first 3 rounds to grasp whats happening
brexit = 6    # in round 6 - 

I thought with these dictionaries the change from dummy matrix to the real data should be easy. 

In [4]:
dummy_playing_field_matrix.shape

(80, 80)

In [5]:

ownership = {
    'Forester1': [11, 12, 21, 22],  # Forester 1 owns Plot 11, 12, 21, 22  
    'Farmer1': [13, 14,23, 24],     # Farmer 1 owns Plot 13, 14, 23, 24 
    'Farmer2': [31, 32, 41, 42],    # Farmer 2 owns Plot 31, 32, 41, 42 
    'Forester2': [33, 34, 43, 44]  # Forester 2 owns Plot 33, 34, 43, 44 
}

landuse = {
    'cattle': 1,
    'sheep': 2,
    'n_forest': 3,
    'c_forest': 4
}

number_of_players = len(ownership) + 1 # +1 because of the tourism 
# for more than 5 players there would be a new way to allocate ownership of plots.

A function for the decision if teamwork takes place.  It assumes 4 players allocated as above and that all have to agree to it. 

In [6]:
# assumes four players only / if the four corner player say yes then it's true.
def teamwork(cooperation_matrix):
    teamwork = False
    row, col = cooperation_matrix.shape
    if cooperation_matrix[0][0] == cooperation_matrix[0][col-1] == cooperation_matrix[row-1][0] == cooperation_matrix[row-1][col-1] == True:
        teamwork = True
    return teamwork

In [7]:
team_work = teamwork(cooperation_matrix)
team_work

True

A function returning the number of landuse-pixel for a given matrix (according to the dictionary landuse above). 

In [8]:
# get the total yield for the current map
def yield_map(field):
    tot_cattle = np.count_nonzero(field == landuse['cattle'])
    tot_sheep = np.count_nonzero(field == landuse['sheep'])
    tot_n_forest = np.count_nonzero(field == landuse['n_forest'])
    tot_c_forest = np.count_nonzero(field == landuse['c_forest'])
    return tot_cattle, tot_sheep, tot_n_forest, tot_c_forest

In [9]:
# get the yield for the current playing field
tot_cattle, tot_sheep, tot_n_forest, tot_c_forest = yield_map(lulc_matrix)

In [10]:
# get the yield for a plot
plot_cattle, plot_sheep, plot_n_forest, plot_c_forest = yield_map(plot_definition_matrix)

In [11]:
# how much agricultural product is consumed (in terms of area)
farming_area_max = 8 # easiest way to address to much production and effect of land degradation (but unrealistic) - for a quick game of a certain amounts of rounds maybe good enough, if the number is not known
c_forest_max = 10
# how much money does a land use change cost you (c:commercial forest, n: native forest, s:sheep, c: cattle)


In [12]:
cattle = []
cattle.append(1)
cattle

[1]

In [13]:
lulc_matrix[0:39][0:39].shape

(39, 80)

In [14]:
# sorry i only noticed later that i used it to "crop" the matrix.
# but it doesn't seem to work...
lulc_matrix[np.ix_([0,40-1],[0,40 - 1])]

array([[1, 1],
       [1, 1]], dtype=uint8)

In [15]:
sheep_pp = 20600/(80*80/4/2.5)
sheep_pp*80*20/2.5

20600.0

In [16]:
def profit_pp(round, tot_sheep, tot_cattle, tot_n_forest, tot_c_forest, cattle_pp = [1], sheep_pp = [0.1], n_forest_pp = [0.1], c_forest_pp = [0.4]):      
    #doesn't take tourism effects into account yet. and the equations are pretty random.
    if round != 0:
        cattle_pp.append(tot_sheep[0] / (1 + tot_cattle[round]*4 + tot_sheep[round])*cattle_pp[0]) # a certain demand - + 1 so that its never going to infinity should all land become forest
        sheep_pp.append(sheep_pp[0]) # assume sheep can go everywhere, eat everything and no degradation and its profit only influences cattle by competition
        c_forest_pp.append((tot_c_forest[0] + tot_n_forest[0])/(1 + tot_c_forest_tot[round] * 3 + area_n_forest_tot[round]) * c_forest_pp[0])
        n_forest_pp.append(n_forest_pp[0]) # assumes native forest can grow everywhere and its profit only influences the commercial forest through competition in the timber market
    return cattle_pp, sheep_pp, n_forest_pp, c_forest_pp


In [17]:
from pdb import set_trace

In [18]:
def money_pp_farmer(round, tourism, teams, brexit, teamwork, area_sheep, area_cattle, area_c_forest, area_n_forest, sheep_pp, cattle_pp, n_forest_pp, c_forest_pp, nf_to_s, nf_to_c, s_to_c, s_to_nf, c_to_s, c_to_nf, subsidies):
    if round == 0: 
        money = area_sheep[0] * sheep_pp[0] 
    else:   
         # costs of landscape change
        try:
            d_sheep = area_sheep[round] - area_sheep[round-1]
        except: set_trace()
        d_cattle = area_cattle[round] - area_cattle[round-1]
        d_n_forest = area_n_forest[round] - area_n_forest[round-1] # necessary to potentially allow two changes (i.e. a rise or native forests and cattle on cost of sheep )
        m_change = 0
        m_brexit = 0
        if d_n_forest < 0: 
            m_change += min([d_cattle, d_n_forest], key= abs)*nf_to_c
            m_change += min([d_sheep, d_n_forest], key= abs)*nf_to_s
        if d_sheep < 0:
            m_change += min([d_cattle, d_sheep], key= abs)*s_to_c
            m_change += min([d_sheep, d_n_forest], key= abs)*s_to_nf 
        if d_cattle < 0:
            m_change += min([d_cattle, d_sheep], key= abs)*c_to_s
            m_change += min([d_cattle, d_n_forest], key= abs)*c_to_nf 
        # money from the area
        m_area = (area_sheep[round] * sheep_pp[round]) + (area_cattle[round] * cattle_pp[round])
        if teamwork == True and teams > round:
            m_teamwork = area_c_forest[round]*c_forest_pp[round] + area_n_forest[round]*n_forest_pp[round]
        if brexit > round:
            m_brexit = (1-subsidies)*(area_sheep[round] * sheep_pp[round])
            if d_n_forest > 0:
                m_brexit += d_n_forest * subsidies
        m_tourism = tourism * m_area
        # maybe return later on the performance of each landuse/industrie --> append() so that its easy to plot?
        money = m_area*t + m_change + m_tourism + m_teamwork + brexit
    
    return money

In [19]:


def money_pp_forester(round, tourism, teams, brexit, teamwork, area_sheep, area_c_forest, area_n_forest, sheep_pp, n_forest_pp, c_forest_pp, nf_to_cf, cf_to_nf, subsidies):

# '''
#     area_c_forest: number of pixel displaying commercial forest
#     area_n_forest: number of pixel displaying native forest
#     round: round of the game (starting at 0)
# '''
    if round == 0: 
        money = area_c_fore[0] * c_forest_pp[0] + area_n_fore[0] * n_forest_pp[0] 
    else:   
        d_n_forest = area_n_forest[round] - area_n_forest[round-1] # necessary to potentially allow two changes (i.e. a rise or native forests and cattle on cost of sheep )
        m_change = 0
        m_brexit = 0
        # ich habe momentan gemacht, dass man nur etwas verkleinern darf!
        if d_n_forest < 0: 
            m_change += d_c_forest * nf_to_cf
        if d_n_forest > 0:
            m_change += d_n_forest * cf_to_nf
        # money from the area
        m_area = (area_n_forest[round] * n_forest_pp[round]) + ((area_c_forest[round] * c_forest_pp[round]))
        if teamwork == True and teams > round:
            m_teamwork = area_sheep[round]*sheep_pp[round]
        if brexit > round:
            if d_n_forest > 0:
                m_brexit = d_n_forest * subsidies + (1-subsidies)*(area_sheep[round] * sheep_pp[round])
        m_tourism = tourism * m_area
        # maybe return later on the performance of each landuse/industrie --> append() so that its easy to plot?
        money = m_area*t + m_change + m_tourism + m_teamwork + brexit
    
    return money

In [20]:
# assumes 5 players at fixed locations     # i don't know how to include unemployment
def main(rounds, teams, brexit, lulc_matrix, tourism_matrix, n_blocks, rows):
    # assumed costs and prices
    cost_of_living = 29600
    
    sheep_profit = 110
    cattle_profit = 155
    c_forest_profit = 200
    n_forest_profit = 50
    #tourism
    GDP_per_capita = 29600
    farm_income = 39000
    #yield per pixel
    sheep_pp = []
    cattle_pp = []
    c_forest_pp = []
    n_forest_pp = []
    tourism = []
    n_pixel, m_pixel = tourism_matrix.shape
    
    s_pp = GDP_per_capita / (rows*rows/n_blocks/2.5)
    sheep_pp.append(s_pp)
    c_pp = s_pp * cattle_profit/sheep_profit
    cattle_pp.append(c_pp)
    cf_pp = s_pp * c_forest_profit/sheep_profit
    c_forest_pp.append(cf_pp)
    nf_pp = s_pp * n_forest_profit/sheep_profit
    n_forest_pp.append(nf_pp)

    subsidies = 0.8
    cf_to_nf = 0.5 
    nf_to_cf = 0.5
    s_to_c = 0.5
    s_to_nf = c_to_nf = 1 # must stay the same (brexit calculations for farmer)
    c_to_s = 0.5
    nf_to_s = -0.1   # assuming farmers can convert native forest to farmland but not commercial forest (sell wood)
    nf_to_c = 0.8    # takes a while until enough grass grows
    
    rounds = rounds + 1
    cattle = []
    sheep = []
    n_forest = []
    c_forest = []
    len_block = int(rows/math.sqrt(n_blocks))
    fa1_cattle = []
    fa1_sheep = []
    fa1_n_forest = []
    fa1_c_forest = []
    fo1_sheep = []
    fo1_n_forest = []
    fo1_c_forest = []
    fa2_cattle = []
    fa2_sheep = []
    fa2_n_forest = []
    fa2_c_forest = []
    fo2_sheep = []
    fo2_n_forest = []
    fo2_c_forest = []
    print(sheep)
    for i in range(rounds):
        tot_cattle, tot_sheep, tot_n_forest, tot_c_forest = yield_map(lulc_matrix)
        sheep.append(tot_sheep)
        cattle.append(tot_cattle)
        n_forest.append(tot_n_forest)
        c_forest.append(tot_c_forest)
        cattle_pp, sheep_pp, n_forest_pp, c_forest_pp = profit_pp(i, tot_sheep, tot_cattle, tot_n_forest, tot_c_forest, cattle_pp, sheep_pp, n_forest_pp, c_forest_pp)     

        
        # forester 1
        cattle, sheep, n_forest, c_forest = yield_map(lulc_matrix[np.ix_([0,len_block - 1], [0,len_block - 1])])
        # fo1_cattle.append(cattle) # no need for that since he can't change his landuse - teamwork doesn't allow it (yet)
        fo1_sheep.append(sheep)
        fo1_n_forest.append(n_forest)
        fo1_c_forest.append(c_forest)
        
        # farmer 1
        cattle, sheep, n_forest, c_forest = yield_map(lulc_matrix[np.ix_([0,len_block - 1],[len_block, rows-1])])
        fa1_cattle.append(cattle)
        fa1_sheep.append(sheep)
        fa1_n_forest.append(n_forest)
        fa1_c_forest.append(c_forest)
        
        # forester 2
        cattle, sheep, n_forest, c_forest = yield_map(lulc_matrix[np.ix_([len_block, rows-1],[0,len_block - 1])])
        # fo2_cattle.append(cattle) # no need for that since he can't change his landuse - teamwork doesn't allow it (yet)
        fo2_sheep.append(sheep)
        fo2_n_forest.append(n_forest)
        fo2_c_forest.append(c_forest)
        
        # farmer 2
        cattle, sheep, n_forest, c_forest = yield_map(lulc_matrix[np.ix_([0,len_block - 1],[len_block, rows-1])])
        fa1_cattle.append(cattle)
        fa1_sheep.append(sheep)
        fa1_n_forest.append(n_forest)
        fa1_c_forest.append(c_forest)
        print(cattle)

        
        # tourism
        cattle, sheep, n_forest, c_forest = yield_map(tourism_matrix)
        t = (sheep*2 + cattle*1.2 + n_forest*2)/n_pixel/n_pixel/(1+c_forest**2)
        

        if i == 0:
            money_fa1 = money_fa2 = money_fo1 = money_fo2 = 10000
            
        # activates the possibilitie for teamwork after a certain round
        money_fa1 = money_pp_farmer(i, t, teams, brexit, team_work, fa1_sheep, fa1_cattle, fa1_c_forest, fa1_n_forest, sheep_pp, cattle_pp, n_forest_pp, c_forest_pp, nf_to_s, nf_to_c, s_to_c, s_to_nf, c_to_s, c_to_nf, subsidies)
        money_fa2 = money_pp_farmer(i, t, teams, brexit, team_work, fa2_sheep, fa2_cattle, fa2_c_forest, fa2_n_forest, sheep_pp, cattle_pp, n_forest_pp, c_forest_pp, nf_to_s, nf_to_c, s_to_c, s_to_nf, c_to_s, c_to_nf, subsidies)
        money_fo1 = money_pp_forester(i, t, teams, brexit, team_work, fo1_sheep, fo1_c_forest, fo1_n_forest, sheep_pp, n_forest_pp, c_forest_pp, nf_to_cf, cf_to_nf,  subsidies)
        money_fo2 = money_pp_forester(i, t, teams, brexit, team_work, fo1_sheep, fo2_c_forest, fo2_n_forest, sheep_pp, n_forest_pp, c_forest_pp, nf_to_cf, cf_to_nf,  subsidies)
            
   
            
        
            
        print(money_fa1, money_fa2, money_fo1, money_fo2)
    return money_fa1, money_fa2, money_fo1, money_fo2
    

In [21]:
a,b,c,d = main(rounds, teams, brexit, lulc_matrix, tourism_matrix, n_blocks, rows)

[]
4


IndexError: list index out of range